In [1]:
import bs4
import requests
import pandas as pd
from glob import glob
import os
import re
from pandas import DataFrame, read_excel, merge

In [2]:
l=0
y=1
pages = range(0,50)
for page in pages:
    res = requests.get("https://www.flipkart.com/search?q=smartphone&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_1_5_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_1_5_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=smartphone&requestId=a814888c-8c96-4aef-b823-a48e0458b78f&as-backfill=on&page="+str(page))
    soup = bs4.BeautifulSoup(res.text,"lxml")
    Product_Name = soup.select("._3wU53n")
    Price = soup.select("._1vC4OE._2rQ-NK")
    Rating = soup.select(".hGSR34")
    RAM = soup.select(".tVe95H:nth-child(1)")
    Display = soup.select(".tVe95H:nth-child(2)")
    Camera = soup.select(".tVe95H:nth-child(3)")
    Battery = soup.select(".tVe95H:nth-child(4)")
    Processor = soup.select(".tVe95H:nth-child(5)")
    details = pd.DataFrame(columns = ("Product_Name","Price","Rating","RAM","Display","Camera","Battery","Processor"))
    for i,j,k,m,n,o,p,q in zip(Product_Name,Price,Rating,RAM,Display,Camera,Battery,Processor):
        details.loc[l] = i.text,j.text,k.text,m.text,n.text,o.text,p.text,q.text;
        l+=1
    #print(details)
    details.to_excel("C:\\Users\\Aparna\\Assignment 3\\data\\FK_SP_Pg_"+str(y)+".xlsx")
    y+=1

In [3]:
# use pwd to find file location and place previous in that location
stock_files = sorted(glob('data/FK_SP_Pg_*.xlsx'))
#stock_files

In [4]:
details_combine = pd.concat((pd.read_excel(file).assign(filename = file)
          for file in stock_files), ignore_index = True)
details_combine.to_excel("D:\\Jupyter Notebook\\Assignment 2\\V2\\FK_SP_Multiple_Pgs_Combined.xlsx")

In [5]:
df = pd.DataFrame(details_combine)

In [6]:
df['Price1']=df['Price'].replace('[^\d.]','',regex=True)

In [7]:
df[['RAM1','ROM','Expnd_Mem']]=df['RAM'].str.split("|",expand=True)

In [8]:
df[['Rear_Camera','Front_Camera']]=df['Camera'].str.split("|",expand=True)

In [9]:
df['Battery1']=df['Battery'].replace('[^\d.]','',regex=True)

In [10]:
df['Processor1']=df['Processor'].str.replace(' Processor','')

In [11]:
df[['del','Disp']]=df['Display'].str.split("(",expand=True)

In [12]:
df[['Display','del1']]=df['Disp'].str.split(")",expand=True)
df['Display']=df['Display'].str.replace(' inch','')

In [13]:
df = df.drop(columns = "del")
df = df.drop(columns = "Disp")
df = df.drop(columns = "del1")

In [14]:
df[['Brand','del2']]=df['Product_Name'].str.split(" ",n=1,expand=True)

In [15]:
df = df.drop(columns = "del2")

In [16]:
df2 = read_excel('Smart_phone_Brand_Names.xlsx')

In [17]:
df3 = df.merge(df2, on='Brand', how='left')

In [18]:
#df3 = df3.drop(columns = "Brand")
#df3 = df3.drop(columns = "Unnamed: 0")

In [19]:
#df3 = df3.drop(columns = "filename")
df3

,Unnamed: 0,Product_Name,Price,Rating,RAM,Display,Camera,Battery,Processor,filename,Price1,RAM1,ROM,Expnd_Mem,Rear_Camera,Front_Camera,Battery1,Processor1,Brand,Brand_Name
0,0,"Realme C11 (Rich Grey, 32 GB)","₹7,499",4.4,2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,6.5,13MP + 2MP | 5MP Front Camera,5000 mAh Battery,Mediatek Helio G35 Processor,data\FK_SP_Pg_1.xlsx,7499,2 GB RAM,32 GB ROM,Expandable Upto 256 GB,13MP + 2MP,5MP Front Camera,5000,Mediatek Helio G35,Realme,Realme
1,0,"Realme C11 (Rich Grey, 32 GB)","₹7,499",4.4,2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,6.5,13MP + 2MP | 5MP Front Camera,5000 mAh Battery,Mediatek Helio G35 Processor,data\FK_SP_Pg_1.xlsx,7499,2 GB RAM,32 GB ROM,Expandable Upto 256 GB,13MP + 2MP,5MP Front Camera,5000,Mediatek Helio G35,Realme,Realme
2,1,"Realme C11 (Rich Green, 32 GB)","₹7,499",4.4,2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,6.5,13MP + 2MP | 5MP Front Camera,5000 mAh Battery,Mediatek Helio G35 Processor,data\FK_SP_Pg_1.xlsx,7499,2 GB RAM,32 GB ROM,Expandable Upto 256 GB,13MP + 2MP,5MP Front Camera,5000,Mediatek Helio G35,Realme,Realme
3,1,"Realme C11 (Rich Green, 32 GB)","₹7,499",4.4,2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,6.5,13MP + 2MP | 5MP Front Camera,5000 mAh Battery,Mediatek Helio G35 Processor,data\FK_SP_Pg_1.xlsx,7499,2 GB RAM,32 GB ROM,Expandable Upto 256 GB,13MP + 2MP,5MP Front Camera,5000,Mediatek Helio G35,Realme,Realme
4,2,"POCO M2 Pro (Green and Greener, 64 GB)","₹14,999",4.5,6 GB RAM | 64 GB ROM | Expandable Upto 512 GB,6.67,48MP + 8MP + 5MP + 2MP | 16MP Front Camera,5000 mAh Lithium-ion Polymer Battery,Qualcomm Snapdragon 720G Processor,data\FK_SP_Pg_1.xlsx,14999,6 GB RAM,64 GB ROM,Expandable Upto 512 GB,48MP + 8MP + 5MP + 2MP,16MP Front Camera,5000,Qualcomm Snapdragon 720G,POCO,POCO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006,213,"Samsung Galaxy A10s (Black, 32 GB)","₹10,888",4.3,3 GB RAM | 32 GB ROM | Expandable Upto 512 GB,6.2,13MP + 2MP | 8MP Front Camera,4000 mAh Lithium-ion Battery,MediaTek MT6762 Processor,data\FK_SP_Pg_9.xlsx,10888,3 GB RAM,32 GB ROM,Expandable Upto 512 GB,13MP + 2MP,8MP Front Camera,4000,MediaTek MT6762,Samsung,Samsung
2007,214,"OPPO F15 (Blazing Blue, 128 GB)","₹16,990",4.3,4 GB RAM | 128 GB ROM | Expandable Upto 256 GB,6.4,48MP + 8MP + 2MP + 2MP | 16MP Front Camera,4000 mAh Battery,MediaTek Helio P70 Processor,data\FK_SP_Pg_9.xlsx,16990,4 GB RAM,128 GB ROM,Expandable Upto 256 GB,48MP + 8MP + 2MP + 2MP,16MP Front Camera,4000,MediaTek Helio P70,OPPO,OPPO
2008,214,"OPPO F15 (Blazing Blue, 128 GB)","₹16,990",4.3,4 GB RAM | 128 GB ROM | Expandable Upto 256 GB,6.4,48MP + 8MP + 2MP + 2MP | 16MP Front Camera,4000 mAh Battery,MediaTek Helio P70 Processor,data\FK_SP_Pg_9.xlsx,16990,4 GB RAM,128 GB ROM,Expandable Upto 256 GB,48MP + 8MP + 2MP + 2MP,16MP Front Camera,4000,MediaTek Helio P70,OPPO,OPPO
2009,215,"LG G8X (Black, 128 GB)","₹70,000",4.6,6 GB RAM | 128 GB ROM | Expandable Upto 2 TB,6.4,12MP + 13MP | 32MP Front Camera,4000 mAh Battery,Qualcomm Snapdragon™ 855 Octa-core (up to 2.84...,data\FK_SP_Pg_9.xlsx,70000,6 GB RAM,128 GB ROM,Expandable Upto 2 TB,12MP + 13MP,32MP Front Camera,4000,Qualcomm Snapdragon™ 855 Octa-core (up to 2.84...,LG,LG


In [20]:
df3.to_excel("D:\\Jupyter Notebook\\Assignment 2\\V2\\FK_SP_Multiple_Pgs_Combined_Cleansed_V1.xlsx")